In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A


from going_modular.model.TripletFaceRecognition import EmbeddingNet, TripletNet
from going_modular.dataloader.triplet import TripletDataset
from going_modular.loss.TripletLoss import TripletLoss
from going_modular.train_eval.triplet.train import fit
from going_modular.dataloader.triplet import CustomExrDataset
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

import os

import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': './Dataset/Albedo/train',
    'test_dir': './Dataset/Albedo/test',
    
    # Cấu hình train
    'epochs': 399,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'embedding_size': 512,
    
    'start_lr': 1e-4,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # triplet
    'margin': 1.,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

triplet_train_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=train_transform, type=CONFIGURATION['type'], train=True)
triplet_test_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=test_transform, type=CONFIGURATION['type'], train=False)

triplet_train_loader = DataLoader(
    triplet_train_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

triplet_test_loader = DataLoader(
    triplet_test_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

roc_train_dataset = CustomExrDataset(CONFIGURATION['train_dir'], transform=train_transform, type=CONFIGURATION['type'])
roc_train_loader = DataLoader(
    roc_train_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

roc_test_dataset = CustomExrDataset(CONFIGURATION['test_dir'], transform=test_transform, type=CONFIGURATION['type'])
roc_test_loader = DataLoader(
    roc_test_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

embedding_net = EmbeddingNet(len_embedding=CONFIGURATION['embedding_size']).to(device)
model = TripletNet(embedding_net).to(device)
criterion = TripletLoss(CONFIGURATION['margin'])
optimizer = Adam(model.parameters(), lr=CONFIGURATION['start_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath('checkpoint/samenetwork/triplet/' + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=50,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=40
)      

checkpoint_path = os.path.abspath('checkpoint/samenetwork/triplet/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

2024-12-21 22:31:01.376249: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-21 22:31:02.551548: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    triplet_train_loader=triplet_train_loader, 
    triplet_test_loader=triplet_test_loader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    epochs=CONFIGURATION['epochs'], 
    device=device,
    roc_train_loader=roc_train_loader, 
    roc_test_loader=roc_test_loader,
    early_max_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

In [2]:
checkpoint = torch.load('checkpoint/samenetwork/triplet/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
print(epoch)

382


In [3]:
fit(
    conf=CONFIGURATION,
    start_epoch=382,
    model=model,
    triplet_train_loader=triplet_train_loader, 
    triplet_test_loader=triplet_test_loader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    epochs=CONFIGURATION['epochs'], 
    device=device,
    roc_train_loader=roc_train_loader, 
    roc_test_loader=roc_test_loader,
    early_max_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 383:
	train: loss: 0.0825 | auc_cos: 0.9776 | auc_eu: 0.9820
	test: loss: 0.0302 | auc_cos: 0.9489 | auc_eu: 0.9534
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/concat/checkpoint/samenetwork/triplet/albedo/models/checkpoint.pth
Epoch 384:
	train: loss: 0.1116 | auc_cos: 0.9691 | auc_eu: 0.9768
	test: loss: 0.0360 | auc_cos: 0.9350 | auc_eu: 0.9492
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/concat/checkpoint/samenetwork/triplet/albedo/models/checkpoint.pth
Epoch 385:
	train: loss: 0.1429 | auc_cos: 0.9687 | auc_eu: 0.9720
	test: loss: 0.0555 | auc_cos: 0.9432 | auc_eu: 0.9469
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/concat/checkpoint/samenetwork/triplet/albedo/models/checkpoint.pth
Epoch 386:
	train: loss: 0.1059 | auc_cos: 0.9638 | auc_eu: 0.9687
	test: loss: 0.0656 | auc_cos: 0.9353 | auc_eu: 0.9408
	Saving model and optimizer state to /media/